## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [4]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='./artifacts_local/0', creation_time=1735065388577, experiment_id='0', last_update_time=1735065388577, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/12/24 18:37:12 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2024/12/24 18:37:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: './artifacts_local/1/4613437fa6c441d49c9b800d4f5154d9/artifacts'
🏃 View run bald-trout-312 at: http://127.0.0.1:5000/#/experiments/1/runs/4613437fa6c441d49c9b800d4f5154d9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='./artifacts_local/1', creation_time=1735065432502, experiment_id='1', last_update_time=1735065432502, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='./artifacts_local/0', creation_time=1735065388577, experiment_id='0', last_update_time=1735065388577, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [9]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [11]:
client.search_registered_models()

[]

In [15]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Search for runs in the experiment with ID '1'
runs = client.search_runs(experiment_ids=['1'], order_by=["attributes.start_time DESC"])

# Get the run_id of the most recent run
if runs:
    run_id = runs[0].info.run_id
    print(f"Run ID: {run_id}")

    # Register the model from the run
    mlflow.register_model(
        model_uri=f"runs:/{run_id}/models",
        name='iris-classifier'
    )
else:
    print("No runs found for experiment ID 1.")


Successfully registered model 'iris-classifier'.
2024/12/24 18:39:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1


Run ID: 4613437fa6c441d49c9b800d4f5154d9


Created version '1' of model 'iris-classifier'.
